# つくりながら学ぶ！深層強化学習 PyTorchによる実践プログラミング

## 迷路問題

In [1]:
using Revise, Pkg
using NaNStatistics, Distributions

Pkg.activate("./SimpleRL")

  Activating project at `d:\github\julia_ml-tuto\04_reinforcement-learning\pytorch\SimpleRL`


In [13]:
using SimpleRL

include("./inc/retype.jl")

"""
勾配降下法モデル
- 強化学習環境 追加インターフェイス:
    - 方策パラメータ取得: () -> (方策パラメータ)
    - 方策パラメータ設定: (方策パラメータ)!
    - 方策マトリクス取得: () -> (方策マトリクス)
- 関数:
    - 方策パラメータ更新: ()!
"""
# Additional Interfaces
get_theta(::Env) = error("get_theta(::Env) -> (theta::Matrix{<:Number}): Get policy parameters of the environment.")
set_theta!(::Env, theta::Matrix{<:Number}) = error("set_theta!(::Env, theta::Matrix{<:Number}): Set policy parameters to the environment.")
get_pi(::Env) = error("get_pi(::Env) -> (pi::Matrix{<:Number}): Get policy matrix of the environment.")

"""
    update_theta!(env::Env; eta::AbstractFloat = 0.1)

方策勾配法による方策パラメータの更新

- `eta::AbstractFloat`: 学習率
"""
update_theta!(env::Env; eta::AbstractFloat = 0.1) = begin
    records = trajectory(env)()
    theta_n = get_theta(env)
    pi_n = get_pi(env)

    # ゴールまでの総ステップ数: ゴール地点のステップは除外
    T = length(records) - 1

    # Δθ の計算
    delta_theta = [
        isnan(theta_n[i, j]) ? NaN : (
            begin
                # 状態 = s_i である記録を取得
                SA_i = filter(SA -> SA.state == i, records)

                # 状態 = s_i で 行動 = a_j をとった記録を取得
                SA_ij = filter(SA -> SA.state == i && SA.action == j, records)

                # N(s_i, a), N(s_i, a_j)
                N_i = length(SA_i)
                N_ij = length(SA_ij)

                # Δθ
                (N_ij + pi_n[i, j] * N_i) / T
            end
        )
        for i = 1:size(theta_n, 1), j = 1:size(theta_n, 2)
    ]

    # θ更新
    theta_next = theta_n .+ eta .* delta_theta
    set_theta!(env, theta_next)
end

update_theta!

In [16]:
using ProgressMeter

# MazeEnv: 勾配降下法に必要なインターフェイスを追加
get_theta(env::MazeEnv) = env.theta
set_theta!(env::MazeEnv, theta::Matrix{<:Number}) = SimpleRL.set_theta!(env, theta)
get_pi(env::MazeEnv) = env.policy.pi

"""
    train!(env::Env; max_epochs::Int = 10_000, stop_epsilon::AbstractFloat = 10^-8) -> (records::Vector{<:NamedTuple})

勾配降下法による学習実行

- `max_epochs::Int`: 最大学習回数
- `stop_epsilon::AbstractFloat`: 学習完了の方策変化しきい値
- `records::Vector{<:NamedTuple}`: `[(delta_pi = 方策変化の絶対値和, n_steps = ゴールまでのステップ数)]`
"""
train!(env::Env; max_epochs::Int = 10_000, stop_epsilon::AbstractFloat = 10^-8) = begin
    # 学習記録: [(方策変化の絶対値和, ゴールまでのステップ数), ...]
    records = NamedTuple[]

    # 方策勾配法により学習ループする
    @showprogress for _ = 1:max_epochs
        n_steps = execute!(env) # シナリオ実行

        pi_n = get_pi(env)
        update_theta!(env) # 方策パラメータ θ の更新
        pi_next = get_pi(env)

        # 方策変化の絶対値和
        delta_pi = sum(abs.(pi_next .- pi_n))
        # 記録: (方策変化の絶対値和, ゴールまでのステップ数)
        push!(records, (delta_pi = delta_pi, n_steps = n_steps))

        # 終了条件
        delta_pi < stop_epsilon && break
    end

    records
end

train!

In [17]:
# 勾配降下法による学習実行
env = MazeEnv()
records = train!(env)

Progress: 100%|█████████████████████████████████████████| Time: 0:22:30


10000-element Vector{NamedTuple}:
 (delta_pi = 0.014882092556085136, n_steps = 14)
 (delta_pi = 0.020658101693908304, n_steps = 4)
 (delta_pi = 0.01087033644340879, n_steps = 24)
 (delta_pi = 0.00998261459094213, n_steps = 10)
 (delta_pi = 0.006813463110062923, n_steps = 34)
 (delta_pi = 0.008111677867489675, n_steps = 4)
 (delta_pi = 0.008857175690487606, n_steps = 20)
 (delta_pi = 0.006768710649111054, n_steps = 28)
 (delta_pi = 0.006257468478895023, n_steps = 20)
 (delta_pi = 0.005863275871530904, n_steps = 26)
 (delta_pi = 0.005780406388977477, n_steps = 10)
 (delta_pi = 0.005662528741111839, n_steps = 14)
 (delta_pi = 0.006067168402568657, n_steps = 6)
 ⋮
 (delta_pi = 3.4459950992715704e-7, n_steps = 4)
 (delta_pi = 3.429859877779329e-7, n_steps = 24)
 (delta_pi = 3.4746977589650285e-7, n_steps = 12)
 (delta_pi = 3.5255309976944815e-7, n_steps = 30)
 (delta_pi = 3.5637635731333717e-7, n_steps = 14)
 (delta_pi = 3.6612606879593557e-7, n_steps = 16)
 (delta_pi = 3.6659546998052406e-